In [1]:
# 建议与 7.17.12 服务器版本相匹配
%pip install elasticsearch==7.17.12

Note: you may need to restart the kernel to use updated packages.


In [1]:
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch(
    hosts=["http://localhost:9200"],  
    request_timeout=30,
    max_retries=10,
    retry_on_timeout=True
)

In [3]:
print(es.info())   

{'name': '254304d0edc0', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'O5_RtdGsRaeimBqE05E4HQ', 'version': {'number': '7.17.12', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'e3b0c3d3c5c130e1dc6d567d6baef1c73eeb2059', 'build_date': '2023-07-20T05:33:33.690180787Z', 'build_snapshot': False, 'lucene_version': '8.11.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}


D:\anaconda\envs\es\Lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [5]:
create_index_body = {
    "settings": {
        "index": {
            "number_of_shards": 3,
            "number_of_replicas": 1
        }
    },
    "mappings": {
        "properties": {
            "chunk": { "type": "text" },
            "doc_id": { "type": "keyword" },
            "chunk_vector": {
                "type": "dense_vector",
                "dims": 1024
            }
        }
    }
}


if not es.indices.exists(index="dd_report_data"):  
    es.indices.create(index="dd_report_data", body=create_index_body)  

In [4]:
pip install FlagEmbedding

Note: you may need to restart the kernel to use updated packages.


In [3]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel("BAAI/bge-m3", use_fp16=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [46]:
def embed(texts: list[str]) -> list[list[float]]:
    out = model.encode(texts,
                    max_length=8192,
                    return_dense=True, return_sparse=False, return_colbert_vecs=False)
    return out["dense_vecs"][0].tolist()  # 1024维

In [19]:
%pip install langchain==0.1.16
%pip install jsonlines   

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=8192,
    chunk_overlap=100,
    separators=["@@SPLIT@@"]
)

In [7]:
def embed(texts: list[str]) -> list[list[float]]:
    
    emb = model.encode(
        texts,
        batch_size=64,
        max_length=4096,      
    )
  
    return emb.tolist()

In [8]:
from pathlib import Path
import uuid

In [9]:
from tqdm import tqdm
import hashlib, uuid, json
import jsonlines, hashlib, datetime, re
from elasticsearch import helpers, exceptions
import random, datetime

In [10]:
TXT_DIR    =  Path(r"C:\Users\sxmxs\Desktop\dd_rapprot")        

In [ ]:
for txt_file in tqdm(list(TXT_DIR.rglob("*.txt")), desc="文件"):
    helpers.bulk(es, gen_actions(txt_file), chunk_size=8192, request_timeout=120)
    print(f"✅ 已写入 {txt_file.relative_to(TXT_DIR.parent)}")

文件:   0%|          | 0/1 [00:00<?, ?it/s]You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [10]:
from transformers import AutoTokenizer
import re
from typing import List, Tuple

HARD_TOKEN = "@@SPLIT@@"
TOKENIZER_ID = "BAAI/bge-large-zh-v1.5"  # 换成你的嵌入模型
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_ID)

def split_by_tokens_keep_text(text: str, max_tokens=8192, overlap_tokens=64) -> List[str]:
    enc = tokenizer(
        text,
        add_special_tokens=False,
        return_offsets_mapping=True,
        return_attention_mask=False,
        return_token_type_ids=False,
    )
    ids = enc["input_ids"]
    offs = enc["offset_mapping"]
    step = max_tokens - overlap_tokens
    out = []
    for start in range(0, len(ids), step):
        end = min(len(ids), start + max_tokens)
        if start >= len(ids):
            break
        s_char = offs[start][0]
        e_char = offs[end-1][1]
        out.append(text[s_char:e_char])  # ← 直接从原文截取，无空格
    return out or [text]

def split_blocks_then_chunks(text: str, max_tokens=8192, overlap_tokens=64) -> List[Tuple[int,int,str,str,str]]:
    """
    返回 [(block_id, part_id, section, chunk_for_embed, chunk_for_display)]
    """
    text = text.replace("\r\n", "\n")
    blocks = [b.strip() for b in re.split(rf"\s*{re.escape(HARD_TOKEN)}\s*", text) if b.strip()]
    out = []
    for bi, blk in enumerate(blocks):
        m = re.match(r"\s*[-•\s]*\*\*(.+?)\*\*\s*[:：]\s*", blk)
        section = (m.group(1).strip() if m else "")
        parts = split_by_tokens_keep_text(blk, max_tokens, overlap_tokens)
        for pj, piece in enumerate(parts):
            # 用于向量的文本里注入 section 提示；展示用保持原文
            piece_for_embed = f"[{section}] {piece}" if section else piece
            out.append((bi, pj, section, piece_for_embed, piece))
    return out


In [11]:
def gen_actions(txt_path: Path):
    text = txt_path.read_text(encoding="utf-8", errors="ignore")
    pieces = split_blocks_then_chunks(text, max_tokens=8192, overlap_tokens=64)
    embed_texts = [p[3] for p in pieces]     # 带 [公司简介] 的文本
    disp_texts  = [p[4] for p in pieces]     # 原文展示文本
    vecs = embed(embed_texts)                # 生成 1024 维向量（与你 mapping 一致）

    doc_uid = txt_path.stem
    for (bi, pj, section, _, disp), vec in zip(pieces, vecs):
        yield {
            "_index": "dd_report_data",
            "_id": f"{doc_uid}#{bi}#{pj}",
            "_source": {
                "doc_id": doc_uid,
                "block_id": bi,
                "part_id": pj,
                "section": section,
                "section_text": section,
                "chunk": disp,              # ← 展示用原文，无空格
                "chunk_vector": vec
            }
        }


In [6]:
es.indices.delete(index="dd_report_data", ignore_unavailable=True)

D:\anaconda\envs\es\Lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True}

In [15]:
create_index_body = {
  "settings": {"number_of_shards": 1, "number_of_replicas": 0},
  "mappings": {
    "properties": {
      "doc_id":       {"type": "keyword"},
      "block_id":     {"type": "integer"},
      "part_id":      {"type": "integer"},
      "section":      {"type": "keyword"},                # 精确值、过滤/加权
      "section_text": {"type": "text"},                   # BM25 匹配“公司简介”
      "chunk":        {"type": "text"},
      "chunk_vector": {"type": "dense_vector", "dims": 1024}
    }
  }
}

if not es.indices.exists(index="dd_report_data"):  
    es.indices.create(index="dd_report_data", body=create_index_body)  

In [16]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_capability(0))  # 例如 (9, 0) 表示 sm_90


True
NVIDIA GeForce RTX 5090 Laptop GPU
(12, 0)


In [17]:
mapping = es.indices.get_mapping(index="dd_report_data")
from pprint import pprint
pprint(mapping)

{'dd_report_data': {'mappings': {'properties': {'block_id': {'type': 'integer'},
                                                'chunk': {'type': 'text'},
                                                'chunk_vector': {'dims': 1024,
                                                                 'type': 'dense_vector'},
                                                'doc_id': {'type': 'keyword'},
                                                'part_id': {'type': 'integer'},
                                                'section': {'type': 'keyword'},
                                                'section_text': {'type': 'text'}}}}}


In [28]:
indices = es.cat.indices(format="json")

for idx in indices:
    print(f"📦 索引名：{idx['index']}")
    print(f"📊 文档数：{idx['docs.count']}")
    print(f"💾 占用磁盘：{idx['store.size']}")
    print("-" * 30)

📦 索引名：.geoip_databases
📊 文档数：0
💾 占用磁盘：250b
------------------------------
📦 索引名：dd_report_data
📊 文档数：163
💾 占用磁盘：2.2mb
------------------------------
📦 索引名：financial_report_data
📊 文档数：71641
💾 占用磁盘：1.4gb
------------------------------


D:\anaconda\envs\es\Lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [29]:
dsl = {
  "size": 20,
  "query": {
    "script_score": {
      "query": {"match_all": {}},
      "script": {
        "source": "cosineSimilarity(params.qv, doc['chunk_vector']) + 1.0",
        "params": {"qv": q_vec}  # q_vec 是一个 1024 维 list
      }
    }
  },
  "_source": True
}

resp = es.search(index="dd_report_data", body=dsl)

NameError: name 'q_vec' is not defined

In [30]:
q_text = "主要策略介绍"
q_vec  = model.encode([q_text])[0].tolist()      

KeyError: 0

In [23]:
def hybrid_query(q_text: str, q_vec: list[float], dense_weight: float = 1.0):
    # 针对短词（<=4 个汉字）把稀疏权重调高、向量权重调低
    is_short = len(q_text) <= 4 and all('\u4e00' <= ch <= '\u9fff' for ch in q_text)
    sparse_boost = 4.0 if is_short else 1.5
    dense_w = 0.5 if is_short else dense_weight

    base_bool = {
        "should": [
            {   # BM25 覆盖正文
                "multi_match": {
                    "query": q_text,
                    "fields": ["chunk^1", "section_text^5"],
                    "type": "most_fields"
                }
            },
            {   # 精确命中 section（如“公司简介”）强力加权
                "term": {"section": {"value": q_text, "boost": sparse_boost}}
            },
            {   # 词组命中也加权
                "match_phrase": {"chunk": {"query": q_text, "boost": 3.0}}
            }
        ],
        "minimum_should_match": 1
    }

    return {
        "function_score": {
            "query": {"bool": base_bool},
            "boost_mode": "sum",
            "score_mode": "sum",
            "functions": [
                {   # 向量分
                    "script_score": {
                        "script": {
                            "source": "cosineSimilarity(params.qv, doc['chunk_vector']) + 1.0",
                            "params": {"qv": q_vec}
                        }
                    },
                    "weight": dense_w
                }
            ]
        }
    }

# 用法（新客户端参数风格）
resp = es.search(
    index="dd_report_data",
    size=20,
    query=hybrid_query(q_text, q_vec),
    _source=["doc_id","block_id","part_id","section","chunk"]
)


D:\anaconda\envs\es\Lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: The vector functions of the form function(query, doc['field']) are deprecated, and the form function(query, 'field') should be used instead. For example, cosineSimilarity(query, doc['field']) is replaced by cosineSimilarity(query, 'field').
  warnings.warn(message, category=ElasticsearchWarning)


RequestError: RequestError(400, 'search_phase_execution_exception', 'runtime error')

In [24]:
for hit in resp["hits"]["hits"]:
    score = hit["_score"]
    source = hit["_source"]
    chunk = source.get("chunk", "...")

    
    print(f"✅ [{score:.4f}] ")
    print(f"📄 {chunk[:300]}...")
    print("-" * 60)

NameError: name 'resp' is not defined

In [21]:
# -*- coding: utf-8 -*-
import os, re, sys, traceback
from typing import List, Dict, Any

# ---------- 依赖 ----------
# pip install "openai>=1.5.0"
from openai import OpenAI
from transformers.utils.versions import require_version
require_version("openai>=1.5.0", "To fix: pip install openai>=1.5.0")

# ---------- OpenAI 兼容客户端 ----------
# 强制本地直连，避免被系统/公司代理劫持到外网导致 502
os.environ.setdefault("NO_PROXY", "127.0.0.1,localhost")

API_PORT = os.getenv("API_PORT", "8000")
BASE_URL = f"http://127.0.0.1:{API_PORT}/v1"
API_KEY  = os.getenv("API_KEY", "0")      # 本地后端通常不校验
PREF_MODEL_ID = os.getenv("MODEL_ID")     # 可选：手动指定模型 id（/v1/models 返回的 id）

# 创建客户端
client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

def pick_model_id(preferred: str | None = None) -> str:
    """优先使用用户指定的 MODEL_ID；否则从 /v1/models 列表里选一个可用 id。"""
    models = client.models.list()
    ids = [m.id for m in models.data] if getattr(models, "data", None) else []
    if not ids:
        raise RuntimeError("后端未返回任何模型：请确认 API 已启动且 /v1/models 可访问。")

    if preferred:
        if preferred in ids:
            return preferred
        else:
            print(f"⚠️ 指定的 MODEL_ID='{preferred}' 不在可用列表中，将改用 '{ids[0]}'")
    return ids[0]

MODEL_ID = pick_model_id(PREF_MODEL_ID)
TIMEOUT_S = float(os.getenv("API_TIMEOUT", "120"))

print(f"✅ 已连接 {BASE_URL} ，使用模型：{MODEL_ID}")

# ---------- Prompt 模板（原样保留） ----------
CLASSIFY_TEMPLATE = """
# role
意图分类器
## profile
根据历史对话对用户输入 query 进行意图分类。

[类别标签以及定义]
1.闲聊 - 不涉及任务、信息或咨询的随意聊天内容
2.擦边 - 涉及敏感词、性暗示、不适宜公开场合的内容
3.政治人物批判 - 包含对政府、官员或政策的负面评论（不包括正常的财经新闻咨询）
4.金融相关咨询 - 包括个股、行业、基金、指标、财务、估值、市场动态、经济数据等金融投资类问题

【参考例子】
query: pussy
label: 擦边

query: 特朗普退休了吗
label: 政治人物批判

query: 茅台股票波动大吗
label: 金融相关咨询

query: 贵州茅台今年财报发布了吗
label: 金融相关咨询

query: 你喜欢什么颜色？
label: 闲聊

[历史对话]（如果无历史请忽略）
{history_dialogue}

【用户query】
{query}

请直接输出 query 的意图结果。
""".strip()

QUERY_REWRITE_TEMPLATE = """
根据上下文历史对话对 query 改写，进行补全/指代/完整的改写，非口语化，关键词提取，便于做检索

【历史对话】最近5轮
{history_dialogue}

【query】
{query}

【参考例子】
- 历史对话
human: 我想问一下a款这个车是前驱还是后驱
bot: 后驱
human: 那b呢
- 输出结果
改写后的query: 那b款的车子是前驱还是后驱?
提取的关键词:b款, 前驱, 后驱, 汽车

请直接输出改写结果。

改写后的query:
提取的关键词:
""".lstrip()

ANSWER_PROMPT_TMPL = """
# 角色：金融咨询助手
- 只回答与金融投资/宏观经济/财报解读相关的问题  
- 回答必须基于【资料块】给出的事实，若无信息请回答 “我不确定”

## 规则
1. 内容健康友好，语言简洁明了，有逻辑
2. 先给结论，再列引用编号；结论之外不要透露推理过程
3. 无法回答时直接说 “我不确定”，不要编造
4. 回答中的任何数字/事实都要能在引用里找到

## 输入
### 资料块（已按相关度排序）
{context_blocks}

### 用户问题
{query}

## 输出格式
答案：<一段到两段直接给出结论>  
引用：<用 ①②③… 标注用到的资料编号，可多个，例如“①③”>
""".strip()

# ---------- OpenAI 版 llm_invoke ----------
def llm_invoke(
    user_prompt: str,
    system_prompt: str = "",
    *,
    temperature: float = 0.2,
    max_new_tokens: int = 512,
    top_p: float = 0.7,
) -> str:
    # OpenAI 参数叫 max_tokens
    EPS = 1e-3
    t = max(float(temperature), EPS)
    p = max(float(top_p), EPS)

    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": user_prompt})

    try:
        resp = client.chat.completions.create(
            model=MODEL_ID,
            messages=messages,
            temperature=t,
            top_p=p,
            max_tokens=int(max_new_tokens),
            timeout=TIMEOUT_S,
        )
        return (resp.choices[0].message.content or "").strip()
    except Exception as e:
        print("❌ 调用聊天接口失败：", e)
        traceback.print_exc()
        return ""

# ---------- 工具函数（原样保留/微调） ----------
def build_history(dialogue: list, max_turns: int = 5) -> str:
    pairs = []
    for m in dialogue[-max_turns*2:]:
        role = "human" if m["role"] == "user" else "bot"
        pairs.append(f"{role}: {m['content']}")
    return "\n".join(pairs) or "（无历史）"

def classify_query(history_dialogue: str, query: str) -> str:
    prompt = CLASSIFY_TEMPLATE.format(history_dialogue=history_dialogue, query=query)
    sys = "只输出一个中文标签：闲聊 / 擦边 / 政治人物批判 / 金融相关咨询。不要附加任何解释或前后缀。"
    out = llm_invoke(prompt, system_prompt=sys, temperature=0.01, max_new_tokens=64, top_p=0.1)
    out = re.sub(r'^\s*(label[:：]?)\s*', '', out or '').strip()
    return out

def _parse_rewrite_output(text: str, fallback_query: str):
    text = text or ""
    m_q  = re.search(r"改写后的?query\s*[:：]\s*(.+)", text)
    m_kw = re.search(r"提取的?关键词\s*[:：]\s*(.+)", text)
    rewrite  = (m_q.group(1).strip() if m_q else fallback_query).strip()
    keywords = [k.strip() for k in (m_kw.group(1).split(","))] if m_kw else []
    rewrite  = rewrite.strip("`").strip()
    keywords = [k for k in keywords if k]
    return rewrite, keywords

def rewrite_query(dialogue: list, query: str):
    hist = build_history(dialogue)
    prompt = QUERY_REWRITE_TEMPLATE.format(history_dialogue=hist, query=query)
    sys = "严格按模板键名输出，不要添加解释。"
    content = llm_invoke(prompt, system_prompt=sys, temperature=0.3, max_new_tokens=256)
    rewrite, keywords = _parse_rewrite_output(content, fallback_query=query)
    return rewrite, keywords, content

def build_context(passages: List[str], top_k: int = 5) -> str:
    items = [p for p in passages[:top_k] if p and p.strip()]
    circled_nums = "①②③④⑤⑥⑦⑧⑨⑩"
    return "\n\n".join(f"{circled_nums[i]} {p}" for i, p in enumerate(items))

def generate_answer(passages: List[str], query: str) -> str:
    context = build_context(passages)
    prompt  = ANSWER_PROMPT_TMPL.format(context_blocks=context, query=query)
    sys = "你是专业的金融咨询助手。"
    return llm_invoke(prompt, system_prompt=sys, temperature=0.2, max_new_tokens=512)

# ---------- （可选）Reranker（保持你的降级逻辑） ----------
# try:
#    from FlagEmbedding import FlagReranker
#    import torch
#    _device = "cuda" if torch.cuda.is_available() else "cpu"
#    reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=_device=="cuda")
# except Exception as e:
#    reranker = None
#    print("⚠️ 未启用 Reranker（FlagEmbedding 未安装或模型不可用）：", e)

def rerank_documents(query: str, initial_docs: List[str], top_n: int = 5) -> List[Dict[str, Any]]:
    if not initial_docs:
        return []
    if reranker is None:
       return [{"score": 0.0, "content": c} for c in initial_docs[:top_n]]
    sentence_pairs = [[query, passage] for passage in initial_docs]
    scores = reranker.compute_score(sentence_pairs)
#   scored = [{"score": float(s), "content": c} for s, c in zip(scores, initial_docs)]
    return sorted(scored, key=lambda x: x["score"], reverse=True)[:top_n]

# ===== 在「Reranker」这一段替换成下面这段 =====
USE_RERANKER = os.getenv("USE_RERANKER", "0") == "1"        # 默认关闭，想开时设置环境变量 USE_RERANKER=1
RERANKER_ID = os.getenv("RERANKER_ID", "BAAI/bge-reranker-v2-m3")
RERANKER_LOCAL_ONLY = os.getenv("RERANKER_LOCAL_ONLY", "1") == "1"  # 默认仅离线加载，避免网络超时

reranker = None
if USE_RERANKER:
    try:
        import torch
        from FlagEmbedding import FlagReranker
        if RERANKER_LOCAL_ONLY:
            os.environ.setdefault("HF_HUB_OFFLINE", "1")  # 没有本地缓存会立刻报错而不是反复重试
        reranker = FlagReranker(
            RERANKER_ID,
            use_fp16=torch.cuda.is_available(),
            local_files_only=RERANKER_LOCAL_ONLY,
        )
        print(f"✅ Reranker 已启用：{RERANKER_ID}")
    except Exception as e:
        reranker = None
        print("⚠️ 未启用 Reranker（降级运行）：", e)
else:
    print("ℹ️ 跳过 Reranker（设置 USE_RERANKER=1 可启用）。")


# ---------- Elasticsearch 示例（与你原来的实现保持一致，占位） ----------
# ---------- Elasticsearch：混合检索（BM25 + 向量 + section 加权） ----------
def _has_section_fields(index_name: str = "dd_report_data") -> bool:
    try:
        props = es.indices.get_mapping(index=index_name)[index_name]["mappings"]["properties"]
        return ("section" in props) or ("section_text" in props)
    except Exception:
        return False

def _hybrid_query_dual(
    q1_text: str, q1_vec: list[float],
    q2_text: str | None, q2_vec: list[float] | None,
    has_section: bool,
    dense_weight: float = 1.0,
):
    """
    q1 = 改写后的 query（主查询，权重更高）
    q2 = 原始用户输入（副查询，权重稍低；为空则不使用）
    """
    # 短查询（<=4汉字）加大稀疏、降低向量
    def _short(q: str) -> bool:
        return len(q) <= 4 and all('\u4e00' <= ch <= '\u9fff' for ch in q)

    is_short_q1 = _short(q1_text)
    dense_w_q1  = 0.5 if is_short_q1 else dense_weight
    sparse_boost_q1 = 4.0 if is_short_q1 else 1.5

    is_short_q2 = _short(q2_text) if q2_text else False
    dense_w_q2  = 0.4 if is_short_q2 else 0.7 * dense_weight
    sparse_boost_q2 = 3.0 if is_short_q2 else 1.2

    fields = ["chunk^1"]
    if has_section:
        fields.append("section_text^5")

    should_clauses = [
        # q1（改写）——主力
        {"multi_match": {"query": q1_text, "fields": fields, "type": "most_fields", "boost": 2.0}},
        {"match_phrase": {"chunk": {"query": q1_text, "boost": 3.0}}},
    ]
    if has_section:
        should_clauses.append({"term": {"section": {"value": q1_text, "boost": sparse_boost_q1}}})

    # q2（原始）——辅助
    if q2_text and q2_text.strip() and q2_text.strip() != q1_text.strip():
        should_clauses.extend([
            {"multi_match": {"query": q2_text, "fields": fields, "type": "most_fields", "boost": 1.2}},
            {"match_phrase": {"chunk": {"query": q2_text, "boost": 2.0}}},
        ])
        if has_section:
            should_clauses.append({"term": {"section": {"value": q2_text, "boost": sparse_boost_q2}}})

    base_bool = {"should": should_clauses, "minimum_should_match": 1}

    functions = [{
        "script_score": {
            "script": {
                "source": "cosineSimilarity(params.qv1, doc['chunk_vector']) + 1.0",
                "params": {"qv1": q1_vec}
            }
        },
        "weight": dense_w_q1
    }]
    if q2_vec is not None:
        functions.append({
            "script_score": {
                "script": {
                    "source": "cosineSimilarity(params.qv2, doc['chunk_vector']) + 1.0",
                    "params": {"qv2": q2_vec}
                }
            },
            "weight": dense_w_q2
        })

    return {
        "function_score": {
            "query": {"bool": base_bool},
            "boost_mode": "sum",
            "score_mode": "sum",
            "functions": functions
        }
    }

def es_hybrid_search(
    rewrite_text: str,
    raw_text: str | None = None,
    index_name: str = "dd_report_data",
    size: int = 20,
):
    # 生成两个向量（相同则只算一次）
    vec1 = model.encode([rewrite_text])[0].tolist()         # noqa: F821
    vec2 = None
    if raw_text and raw_text.strip() and raw_text.strip() != rewrite_text.strip():
        vec2 = model.encode([raw_text])[0].tolist()         # noqa: F821

    has_section = _has_section_fields(index_name)
    query_body = _hybrid_query_dual(
        q1_text=rewrite_text, q1_vec=vec1,
        q2_text=raw_text, q2_vec=vec2,
        has_section=has_section,
        dense_weight=1.0
    )

    return es.search(                                      # noqa: F821
        index=index_name,
        size=size,
        query=query_body,
        _source=["doc_id","block_id","part_id","section","section_text","chunk"]
    )



# ---------- 交互主循环（无联网搜索） ----------
def main():
    print("✅ 就绪：输入你的问题，输入 exit 退出。")
    conversation: List[Dict[str, str]] = []
    turn = 0

    while True:
        try:
            user_input = input("User: ").strip()
        except (EOFError, KeyboardInterrupt):
            print("\nBye!")
            break

        if user_input.lower() == "exit":
            print("Bye!")
            break
        if not user_input:
            continue

        turn += 1
        print("\n" + "─" * 80)
        print(f"🗣️ 第 {turn} 轮 - 用户问题：{user_input}", flush=True)
        print("─" * 80 + "\n")

        conversation.append({"role": "user", "content": user_input})

        # 1) 分类
        history_text = "\n".join([f"{m['role']}: {m['content']}" for m in conversation[-6:-1]])
        label = classify_query(history_text, user_input)
        print("📌 分类结果:", label)

        if "金融" not in label:
            print("❌ 非金融问题，不予处理")
            conversation.append({"role": "assistant", "content": "对不起，我只能处理金融相关问题。"})
            continue

        # 2) 改写
        rewrite, keywords, raw = rewrite_query(conversation, user_input)
        print("✍️ 改写后的 Query:", rewrite)
        print("🔑 提取的关键词:", keywords)

        # 3) 从 ES 召回
        try:
            resp = es_hybrid_search(
                        rewrite_text=rewrite,
                        raw_text=user_input,                  # ← 同时用原始输入
                        index_name="dd_report_data",
                        size=20)
            kb_ctx = [h["_source"].get("chunk","") for h in resp["hits"]["hits"]]

        except Exception as e:
            print("⚠️ ES 查询失败，按无本地知识处理：", e)
            kb_ctx = []

        if kb_ctx:
            print(f"📚 来自 ES 的候选段落：{len(kb_ctx)} 条")
        else:
            print("📚 ES 暂无命中")

        # 4) 重排序（可选）
        merged   = kb_ctx
        top_docs = rerank_documents(rewrite, merged, top_n=10)
        passages_for_answer = [d["content"] for d in top_docs] if top_docs else merged

        # 5) 生成答案（只用本地知识）
        answer = generate_answer(passages_for_answer, query=rewrite)
        print("\n🧠 模型回答：\n", answer)

        conversation.append({"role": "assistant", "content": answer})




✅ 已连接 http://127.0.0.1:8000/v1 ，使用模型：gpt-3.5-turbo
ℹ️ 跳过 Reranker（设置 USE_RERANKER=1 可启用）。


In [74]:
main()

✅ 就绪：输入你的问题，输入 exit 退出。


User:  exit


Bye!


In [27]:
# ====================== Gradio UI ======================
import gradio as gr
import datetime as _dt

DEFAULT_INDEX = "dd_report_data"

def _history_to_dialogue(chat_history: list[tuple[str, str]]) -> list[Dict[str, str]]:
    """Gradio Chatbot 的历史 (user, assistant) -> 你现有函数使用的 [{'role', 'content'}...]"""
    conv = []
    for u, a in chat_history:
        if u:
            conv.append({"role": "user", "content": u})
        if a:
            conv.append({"role": "assistant", "content": a})
    return conv

def _format_hits_md(resp, topn: int = 8) -> str:
    if not resp or "hits" not in resp or not resp["hits"]["hits"]:
        return "（无命中）"
    lines = []
    for i, h in enumerate(resp["hits"]["hits"][:topn], 1):
        sc = h.get("_score", 0.0)
        src = h.get("_source", {})
        sec = src.get("section") or src.get("section_text") or "—"
        txt = (src.get("chunk") or "").strip().replace("\r\n", "\n")
        snippet = (txt[:400] + "…") if len(txt) > 400 else txt
        lines.append(f"**{i:02d}. [{sc:.3f}] {sec}**\n\n{snippet}")
    return "\n\n---\n\n".join(lines)

def respond(message, chat_history, temperature, n_ctx, index_name):
    """
    Gradio 回调：输入 message 和历史，返回（更新后的历史，命中文档、调试信息）
    """
    # 1) 历史转换 + 分类
    conversation = _history_to_dialogue(chat_history)
    history_text = "\n".join([f"{m['role']}: {m['content']}" for m in conversation[-6:]])
    label = classify_query(history_text, message)

    if "金融" not in label:
        assistant = "对不起，我只能处理金融相关问题。"
        chat_history = chat_history + [(message, assistant)]
        debug_md = f"**分类**：{label}  \n**时间**：{_dt.datetime.now()}"
        return chat_history, "（无检索）", debug_md

    # 2) 改写
    rewrite, keywords, _raw = rewrite_query(conversation + [{"role": "user", "content": message}], message)

    # 3) 检索（双查询：原始 + 改写）
    try:
        resp = es_hybrid_search(              # ← 你上面定义的新函数
            rewrite_text=rewrite,
            raw_text=message,
            index_name=index_name,
            size=n_ctx
        )
        kb_ctx = [h["_source"].get("chunk", "") for h in resp["hits"]["hits"]]
        hits_md = _format_hits_md(resp, topn=min(8, n_ctx))
    except Exception as e:
        kb_ctx, hits_md = [], f"检索失败：{e!r}"

    # 4) （可选）重排序
    merged   = kb_ctx
    top_docs = rerank_documents(rewrite, merged, top_n=min(10, len(merged)))
    passages_for_answer = [d["content"] for d in top_docs] if top_docs else merged

    # 5) 生成
    _old_temp = 0.2
    assistant = generate_answer(passages_for_answer[:5], query=rewrite)  # 只取前5段入上下文
    chat_history = chat_history + [(message, assistant)]

    # 6) 调试信息
    debug_md = (
        f"**分类**：{label}\n\n"
        f"**原始问题**：{message}\n\n"
        f"**改写后**：{rewrite}\n\n"
        f"**关键词**：{', '.join(keywords) if keywords else '—'}\n\n"
        f"**时间**：{_dt.datetime.now()}"
    )
    return chat_history, hits_md, debug_md

with gr.Blocks(title="尽调问答助手", theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 尽调问答助手\n输入问题，我会基于本地知识库检索并回答。")

    with gr.Row():
        chatbot = gr.Chatbot(height=420, show_copy_button=True, label="对话")
        with gr.Column():
            hits = gr.Markdown(label="命中文档（Top）", value="（等待检索…）")
            dbg  = gr.Markdown(label="调试信息", value="—")

    with gr.Row():
        msg = gr.Textbox(placeholder="例如：公司简介 / 风控团队职责 / 交易系统架构…", scale=6)
        send = gr.Button("发送", variant="primary", scale=1)
        clear = gr.Button("清空", scale=1)

    with gr.Accordion("高级设置", open=False):
        temperature = gr.Slider(0.0, 1.5, value=0.2, step=0.05, label="生成温度")
        n_ctx = gr.Slider(3, 30, value=20, step=1, label="召回条数")
        index_name = gr.Textbox(value=DEFAULT_INDEX, label="ES 索引名")

    def _submit(user_msg, chat_hist, temperature, n_ctx, index_name):
        return respond(user_msg, chat_hist, temperature, int(n_ctx), index_name)

    send.click(_submit, [msg, chatbot, temperature, n_ctx, index_name], [chatbot, hits, dbg]) \
        .then(lambda: "", None, msg)
    msg.submit(_submit, [msg, chatbot, temperature, n_ctx, index_name], [chatbot, hits, dbg]) \
        .then(lambda: "", None, msg)
    clear.click(lambda: ([], "（已清空）", "—"), None, [chatbot, hits, dbg])

# 作为脚本启动
if __name__ == "__main__":
    # 如果需要公网访问可传 share=True；服务器环境可改 server_name="0.0.0.0"
    demo.queue().launch(server_name="127.0.0.1", server_port=7861, share=False)


C:\Users\sxmxs\AppData\Local\Temp\ipykernel_9012\1948768807.py:85: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=420, show_copy_button=True, label="对话")


OSError: Cannot find empty port in range: 7861-7861. You can specify a different port by setting the GRADIO_SERVER_PORT environment variable or passing the `server_port` parameter to `launch()`.

In [51]:
q_text = "主要策略介绍"
q_vec = embed([q_text])

In [43]:
q_text = "徐玮"

# 方式A：通用 encode（记得 is_query=True）
out = model.encode([q_text])
q_vec = out["dense_vecs"][0].tolist()

In [37]:
def hybrid_query(q_text: str, q_vec: list[float], dense_weight: float = 1.0):
    # 针对短词（<=4 个汉字）把稀疏权重调高、向量权重调低
    is_short = len(q_text) <= 4 and all('\u4e00' <= ch <= '\u9fff' for ch in q_text)
    sparse_boost = 4.0 if is_short else 1.5
    dense_w = 0.5 if is_short else dense_weight

    base_bool = {
        "should": [
            {   # BM25 覆盖正文
                "multi_match": {
                    "query": q_text,
                    "fields": ["chunk^1", "section_text^5"],
                    "type": "most_fields"
                }
            },
            {   # 精确命中 section（如“公司简介”）强力加权
                "term": {"section": {"value": q_text, "boost": sparse_boost}}
            },
            {   # 词组命中也加权
                "match_phrase": {"chunk": {"query": q_text, "boost": 3.0}}
            }
        ],
        "minimum_should_match": 1
    }

    return {
        "function_score": {
            "query": {"bool": base_bool},
            "boost_mode": "sum",
            "score_mode": "sum",
            "functions": [
                {   # 向量分
                    "script_score": {
                        "script": {
                            "source": "cosineSimilarity(params.qv, doc['chunk_vector']) + 1.0",
                            "params": {"qv": q_vec}
                        }
                    },
                    "weight": dense_w
                }
            ]
        }
    }

In [53]:
resp = es.search(
    index="dd_report_data",
    size=20,
    query=hybrid_query(q_text, q_vec),
    _source=["doc_id","block_id","part_id","section","chunk"]
)


In [54]:
for hit in resp["hits"]["hits"]:
    score = hit["_score"]
    source = hit["_source"]
    chunk = source.get("chunk", "...")

    
    print(f"✅ [{score:.4f}] ")
    print(f"📄 {chunk[:300]}...")
    print("-" * 60)

✅ [65.3692] 
📄 - **公司主要投资策略**：1.量化股票策略：
（1）指数增强策略：基金在进行指数化投资的过程中，为试图获得超越指数的投资回报，在被动跟踪指数的基础上，加入增强型的积极投资手段（对个股、行业、风格进行适当调节），对投资组合中持股的行业轮动及个股增强进行适当调整，力求在利用量化模型控制风险的同时获取积极的市场收益。目前宽投的指增产品线已覆盖中证500、中证1000、沪深300等主流指数。
（2）量化选股策略：主要利用数学模型和算法筛选股票、构建投资组合，以做多为主并追求超额收益（Alpha）的策略，核心是通过数据分析和统计模型捕捉市场中的规律性机会。选股范围并不拘束于某一指数，投资组合也不针对...
------------------------------------------------------------
✅ [48.7821] 
📄 - **公司策略有效/失效的主要影响因素**：1、市场结构：量化模型基于历史数据构建，在市场结构稳定时可正常发挥，若市场结构突变（如政策冲击、黑天鹅事件），策略可能表现不佳。
2、市场有效性：新兴市场或散户占比较高的市场（如A股），信息不对称机会多，Alpha易获取。若竞争加剧，多家机构使用相似因子（如动量、估值），易导致策略拥挤，收益被摊薄。
3、市场流动性与波动性：高流动性市场且正常波动环境，便于策略快速调仓，减少滑点损耗，量价因子有效性较高。当市场剧烈波动时噪声干扰增加，模型可能过拟合。特别是在单边暴跌或暴涨行情中，量化模型可能无法有效应对非线性波动（如流动性枯竭）。
4、模型复杂程度：...
------------------------------------------------------------
✅ [36.0890] 
📄 - **子公司及主要关联方**：无...
------------------------------------------------------------
✅ [34.2861] 
📄 - **公司简介**：上海宽投资产管理有限公司成立于2014年12月，2015年4月通过中国证券投资基金业协会备案，2017年4月成为协会会员单位，2018年4月获得投顾资格，2021年资产管理规模突破100亿元。公司专注于国内二级市场量化投资交易，投研团

In [40]:
EMBED_MAX_LEN = 8192
def embed_dense(texts: list[str], max_len: int = EMBED_MAX_LEN) -> list[list[float]]:
    """返回 list[list[float]]；自动过滤空文本；输出 1024 维 python float。"""
    clean = [t for t in texts if isinstance(t, str) and t.strip()]
    if not clean:
        return []
    out = embedder.encode(
        clean,
        max_length=max_len,
        return_dense=True, return_sparse=False, return_colbert_vecs=False,
        normalize_embeddings=True,
    )
    vecs = out["dense_vecs"]
    try:
        vecs = vecs.tolist()
    except Exception:
        pass
    return [[float(x) for x in v] for v in vecs]

In [55]:
# -*- coding: utf-8 -*-
import os, re, sys, traceback
from typing import List, Dict, Any

# ---------- 依赖 ----------
# pip install "openai>=1.5.0"
from openai import OpenAI
from transformers.utils.versions import require_version
require_version("openai>=1.5.0", "To fix: pip install openai>=1.5.0")

# ---------- OpenAI 兼容客户端 ----------
# 强制本地直连，避免被系统/公司代理劫持到外网导致 502
os.environ.setdefault("NO_PROXY", "127.0.0.1,localhost")

API_PORT = os.getenv("API_PORT", "8000")
BASE_URL = f"http://127.0.0.1:{API_PORT}/v1"
API_KEY  = os.getenv("API_KEY", "0")      # 本地后端通常不校验
PREF_MODEL_ID = os.getenv("MODEL_ID")     # 可选：手动指定模型 id（/v1/models 返回的 id）

# 创建客户端
client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

def pick_model_id(preferred: str | None = None) -> str:
    """优先使用用户指定的 MODEL_ID；否则从 /v1/models 列表里选一个可用 id。"""
    models = client.models.list()
    ids = [m.id for m in models.data] if getattr(models, "data", None) else []
    if not ids:
        raise RuntimeError("后端未返回任何模型：请确认 API 已启动且 /v1/models 可访问。")

    if preferred:
        if preferred in ids:
            return preferred
        else:
            print(f"⚠️ 指定的 MODEL_ID='{preferred}' 不在可用列表中，将改用 '{ids[0]}'")
    return ids[0]

MODEL_ID = pick_model_id(PREF_MODEL_ID)
TIMEOUT_S = float(os.getenv("API_TIMEOUT", "120"))

print(f"✅ 已连接 {BASE_URL} ，使用模型：{MODEL_ID}")

# ---------- Prompt 模板（原样保留） ----------
CLASSIFY_TEMPLATE = """
# role
意图分类器
## profile
根据历史对话对用户输入 query 进行意图分类。

[类别标签以及定义]
1.闲聊 - 不涉及任务、信息或咨询的随意聊天内容
2.擦边 - 涉及敏感词、性暗示、不适宜公开场合的内容
3.政治人物批判 - 包含对政府、官员或政策的负面评论（不包括正常的财经新闻咨询）
4.金融相关咨询 - 包括个股、行业、基金、指标、财务、估值、市场动态、经济数据等金融投资类问题

【参考例子】
query: pussy
label: 擦边

query: 特朗普退休了吗
label: 政治人物批判

query: 茅台股票波动大吗
label: 金融相关咨询

query: 贵州茅台今年财报发布了吗
label: 金融相关咨询

query: 你喜欢什么颜色？
label: 闲聊

[历史对话]（如果无历史请忽略）
{history_dialogue}

【用户query】
{query}

请直接输出 query 的意图结果。
""".strip()

QUERY_REWRITE_TEMPLATE = """
根据上下文历史对话对 query 改写，进行补全/指代/完整的改写，非口语化，关键词提取，便于做检索

【历史对话】最近5轮
{history_dialogue}

【query】
{query}

【参考例子】
- 历史对话
human: 我想问一下a款这个车是前驱还是后驱
bot: 后驱
human: 那b呢
- 输出结果
改写后的query: 那b款的车子是前驱还是后驱?
提取的关键词:b款, 前驱, 后驱, 汽车

请直接输出改写结果。

改写后的query:
提取的关键词:
""".lstrip()

ANSWER_PROMPT_TMPL = """
# 角色：金融咨询助手
- 只回答与金融投资/宏观经济/财报解读相关的问题  
- 回答必须基于【资料块】给出的事实，若无信息请回答 “我不确定”

## 规则
1. 内容健康友好，语言简洁明了，有逻辑
2. 先给结论，再列引用编号；结论之外不要透露推理过程
3. 无法回答时直接说 “我不确定”，不要编造
4. 回答中的任何数字/事实都要能在引用里找到

## 输入
### 资料块（已按相关度排序）
{context_blocks}

### 用户问题
{query}

## 输出格式
答案：<一段到两段直接给出结论>  
引用：<用 ①②③… 标注用到的资料编号，可多个，例如“①③”>
""".strip()

# ---------- OpenAI 版 llm_invoke ----------
def llm_invoke(
    user_prompt: str,
    system_prompt: str = "",
    *,
    temperature: float = 0.2,
    max_new_tokens: int = 512,
    top_p: float = 0.7,
) -> str:
    # OpenAI 参数叫 max_tokens
    EPS = 1e-3
    t = max(float(temperature), EPS)
    p = max(float(top_p), EPS)

    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": user_prompt})

    try:
        resp = client.chat.completions.create(
            model=MODEL_ID,
            messages=messages,
            temperature=t,
            top_p=p,
            max_tokens=int(max_new_tokens),
            timeout=TIMEOUT_S,
        )
        return (resp.choices[0].message.content or "").strip()
    except Exception as e:
        print("❌ 调用聊天接口失败：", e)
        traceback.print_exc()
        return ""

# ---------- 工具函数（原样保留/微调） ----------
def build_history(dialogue: list, max_turns: int = 5) -> str:
    pairs = []
    for m in dialogue[-max_turns*2:]:
        role = "human" if m["role"] == "user" else "bot"
        pairs.append(f"{role}: {m['content']}")
    return "\n".join(pairs) or "（无历史）"

def classify_query(history_dialogue: str, query: str) -> str:
    prompt = CLASSIFY_TEMPLATE.format(history_dialogue=history_dialogue, query=query)
    sys = "只输出一个中文标签：闲聊 / 擦边 / 政治人物批判 / 金融相关咨询。不要附加任何解释或前后缀。"
    out = llm_invoke(prompt, system_prompt=sys, temperature=0.01, max_new_tokens=64, top_p=0.1)
    out = re.sub(r'^\s*(label[:：]?)\s*', '', out or '').strip()
    return out

def _parse_rewrite_output(text: str, fallback_query: str):
    text = text or ""
    m_q  = re.search(r"改写后的?query\s*[:：]\s*(.+)", text)
    m_kw = re.search(r"提取的?关键词\s*[:：]\s*(.+)", text)
    rewrite  = (m_q.group(1).strip() if m_q else fallback_query).strip()
    keywords = [k.strip() for k in (m_kw.group(1).split(","))] if m_kw else []
    rewrite  = rewrite.strip("`").strip()
    keywords = [k for k in keywords if k]
    return rewrite, keywords

def rewrite_query(dialogue: list, query: str):
    hist = build_history(dialogue)
    prompt = QUERY_REWRITE_TEMPLATE.format(history_dialogue=hist, query=query)
    sys = "严格按模板键名输出，不要添加解释。"
    content = llm_invoke(prompt, system_prompt=sys, temperature=0.3, max_new_tokens=256)
    rewrite, keywords = _parse_rewrite_output(content, fallback_query=query)
    return rewrite, keywords, content

def build_context(passages: List[str], top_k: int = 5) -> str:
    items = [p for p in passages[:top_k] if p and p.strip()]
    circled_nums = "①②③④⑤⑥⑦⑧⑨⑩"
    return "\n\n".join(f"{circled_nums[i]} {p}" for i, p in enumerate(items))

def generate_answer(passages: List[str], query: str, temperature: float = 0.2, max_new_tokens: int = 512) -> str:
    context = build_context(passages)
    prompt  = ANSWER_PROMPT_TMPL.format(context_blocks=context, query=query)
    sys = "你是专业的金融咨询助手。"
    return llm_invoke(
        prompt,
        system_prompt=sys,
        temperature=float(temperature),
        max_new_tokens=int(max_new_tokens),
        top_p=0.9,
    )


# ---------- （可选）Reranker（保持你的降级逻辑） ----------
# try:
#    from FlagEmbedding import FlagReranker
#    import torch
#    _device = "cuda" if torch.cuda.is_available() else "cpu"
#    reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=_device=="cuda")
# except Exception as e:
#    reranker = None
#    print("⚠️ 未启用 Reranker（FlagEmbedding 未安装或模型不可用）：", e)

def rerank_documents(query: str, initial_docs: List[str], top_n: int = 5) -> List[Dict[str, Any]]:
    if not initial_docs:
        return []
    if reranker is None:
       return [{"score": 0.0, "content": c} for c in initial_docs[:top_n]]
    sentence_pairs = [[query, passage] for passage in initial_docs]
    scores = reranker.compute_score(sentence_pairs)
#   scored = [{"score": float(s), "content": c} for s, c in zip(scores, initial_docs)]
    return sorted(scored, key=lambda x: x["score"], reverse=True)[:top_n]

# ===== 在「Reranker」这一段替换成下面这段 =====
USE_RERANKER = os.getenv("USE_RERANKER", "0") == "1"        # 默认关闭，想开时设置环境变量 USE_RERANKER=1
RERANKER_ID = os.getenv("RERANKER_ID", "BAAI/bge-reranker-v2-m3")
RERANKER_LOCAL_ONLY = os.getenv("RERANKER_LOCAL_ONLY", "1") == "1"  # 默认仅离线加载，避免网络超时

reranker = None
if USE_RERANKER:
    try:
        import torch
        from FlagEmbedding import FlagReranker
        if RERANKER_LOCAL_ONLY:
            os.environ.setdefault("HF_HUB_OFFLINE", "1")  # 没有本地缓存会立刻报错而不是反复重试
        reranker = FlagReranker(
            RERANKER_ID,
            use_fp16=torch.cuda.is_available(),
            local_files_only=RERANKER_LOCAL_ONLY,
        )
        print(f"✅ Reranker 已启用：{RERANKER_ID}")
    except Exception as e:
        reranker = None
        print("⚠️ 未启用 Reranker（降级运行）：", e)
else:
    print("ℹ️ 跳过 Reranker（设置 USE_RERANKER=1 可启用）。")


# ---------- Elasticsearch 示例（与你原来的实现保持一致，占位） ----------
# ---------- Elasticsearch：混合检索（BM25 + 向量 + section 加权） ----------
def _has_section_fields(index_name: str = "dd_report_data") -> bool:
    try:
        props = es.indices.get_mapping(index=index_name)[index_name]["mappings"]["properties"]
        return ("section" in props) or ("section_text" in props)
    except Exception:
        return False

def _hybrid_query_dual(
    q1_text: str, q1_vec: list[float],
    q2_text: str | None, q2_vec: list[float] | None,
    has_section: bool,
    dense_weight: float = 1.0,
):
    """
    q1 = 改写后的 query（主查询，权重更高）
    q2 = 原始用户输入（副查询，权重稍低；为空则不使用）
    """
    # 短查询（<=4汉字）加大稀疏、降低向量
    def _short(q: str) -> bool:
        return len(q) <= 4 and all('\u4e00' <= ch <= '\u9fff' for ch in q)

    is_short_q1 = _short(q1_text)
    dense_w_q1  = 0.5 if is_short_q1 else dense_weight
    sparse_boost_q1 = 4.0 if is_short_q1 else 1.5

    is_short_q2 = _short(q2_text) if q2_text else False
    dense_w_q2  = 0.4 if is_short_q2 else 0.7 * dense_weight
    sparse_boost_q2 = 3.0 if is_short_q2 else 1.2

    fields = ["chunk^1"]
    if has_section:
        fields.append("section_text^5")

    should_clauses = [
        # q1（改写）——主力
        {"multi_match": {"query": q1_text, "fields": fields, "type": "most_fields", "boost": 2.0}},
        {"match_phrase": {"chunk": {"query": q1_text, "boost": 3.0}}},
    ]
    if has_section:
        should_clauses.append({"term": {"section": {"value": q1_text, "boost": sparse_boost_q1}}})

    # q2（原始）——辅助
    if q2_text and q2_text.strip() and q2_text.strip() != q1_text.strip():
        should_clauses.extend([
            {"multi_match": {"query": q2_text, "fields": fields, "type": "most_fields", "boost": 1.2}},
            {"match_phrase": {"chunk": {"query": q2_text, "boost": 2.0}}},
        ])
        if has_section:
            should_clauses.append({"term": {"section": {"value": q2_text, "boost": sparse_boost_q2}}})

    base_bool = {"should": should_clauses, "minimum_should_match": 1}

    functions = [{
        "script_score": {
            "script": {
                "source": "cosineSimilarity(params.qv1, doc['chunk_vector']) + 1.0",
                "params": {"qv1": q1_vec}
            }
        },
        "weight": dense_w_q1
    }]
    if q2_vec is not None:
        functions.append({
            "script_score": {
                "script": {
                    "source": "cosineSimilarity(params.qv2, doc['chunk_vector']) + 1.0",
                    "params": {"qv2": q2_vec}
                }
            },
            "weight": dense_w_q2
        })

    return {
        "function_score": {
            "query": {"bool": base_bool},
            "boost_mode": "sum",
            "score_mode": "sum",
            "functions": functions
        }
    }

def es_hybrid_search(
    rewrite_text: str,
    raw_text: str | None = None,
    index_name: str = "dd_report_data",
    size: int = 20,
):
    # 生成两个向量（相同则只算一次）
    vec1 = embed([rewrite_text])
    vec2 = None
    if raw_text and raw_text.strip() and raw_text.strip() != rewrite_text.strip():
        vec2 = embed_dense([raw_text])[0]

    has_section = _has_section_fields(index_name)
    query_body = _hybrid_query_dual(
        q1_text=rewrite_text, q1_vec=vec1,
        q2_text=raw_text, q2_vec=vec2,
        has_section=has_section,
        dense_weight=1.0
    )

    return es.search(                                      # noqa: F821
        index=index_name,
        size=size,
        query=query_body,
        _source=["doc_id","block_id","part_id","section","section_text","chunk"]
    )



# ---------- 交互主循环（无联网搜索） ----------
def main():
    print("✅ 就绪：输入你的问题，输入 exit 退出。")
    conversation: List[Dict[str, str]] = []
    turn = 0

    while True:
        try:
            user_input = input("User: ").strip()
        except (EOFError, KeyboardInterrupt):
            print("\nBye!")
            break

        if user_input.lower() == "exit":
            print("Bye!")
            break
        if not user_input:
            continue

        turn += 1
        print("\n" + "─" * 80)
        print(f"🗣️ 第 {turn} 轮 - 用户问题：{user_input}", flush=True)
        print("─" * 80 + "\n")

        conversation.append({"role": "user", "content": user_input})

        # 1) 分类
        history_text = "\n".join([f"{m['role']}: {m['content']}" for m in conversation[-6:-1]])
        label = classify_query(history_text, user_input)
        print("📌 分类结果:", label)

        if "金融" not in label:
            print("❌ 非金融问题，不予处理")
            conversation.append({"role": "assistant", "content": "对不起，我只能处理金融相关问题。"})
            continue

        # 2) 改写
        rewrite, keywords, raw = rewrite_query(conversation, user_input)
        print("✍️ 改写后的 Query:", rewrite)
        print("🔑 提取的关键词:", keywords)

        # 3) 从 ES 召回
        try:
            resp = es_hybrid_search(
                        rewrite_text=rewrite,
                        raw_text=user_input,                  # ← 同时用原始输入
                        index_name="dd_report_data",
                        size=20)
            kb_ctx = [h["_source"].get("chunk","") for h in resp["hits"]["hits"]]

        except Exception as e:
            print("⚠️ ES 查询失败，按无本地知识处理：", e)
            kb_ctx = []

        if kb_ctx:
            print(f"📚 来自 ES 的候选段落：{len(kb_ctx)} 条")
        else:
            print("📚 ES 暂无命中")

        # 4) 重排序（可选）
        merged   = kb_ctx
        top_docs = rerank_documents(rewrite, merged, top_n=10)
        passages_for_answer = [d["content"] for d in top_docs] if top_docs else merged

        # 5) 生成答案（只用本地知识）
        answer = generate_answer(passages_for_answer, query=rewrite)
        print("\n🧠 模型回答：\n", answer)

        conversation.append({"role": "assistant", "content": answer})




✅ 已连接 http://127.0.0.1:8000/v1 ，使用模型：gpt-3.5-turbo
ℹ️ 跳过 Reranker（设置 USE_RERANKER=1 可启用）。


In [56]:
main()

✅ 就绪：输入你的问题，输入 exit 退出。


User:  公司简介



────────────────────────────────────────────────────────────────────────────────
🗣️ 第 1 轮 - 用户问题：公司简介
────────────────────────────────────────────────────────────────────────────────

📌 分类结果: 金融相关咨询
✍️ 改写后的 Query: 公司简介
🔑 提取的关键词: []
📚 来自 ES 的候选段落：20 条

🧠 模型回答：
 上海宽投资产管理有限公司成立于2014年12月，2015年4月通过中国证券投资基金业协会备案，2017年4月成为协会会员单位，2018年4月获得投顾资格，2021年资产管理规模突破100亿元。公司专注于国内二级市场量化投资交易，投研团队由具备国内外知名高校统计学、数学、金融工程、计算机科学等专业的硕士或博士学位的顶尖量化研究人才构成，是国内最早涉足量化投资的团队之一，对于量化分析方法和量化投资策略的应用具备丰富的实盘经验，致力于为投资者持续创造稳健的投资收益。公司产品线包括指数增强、量化选股、期货CTA等多策略体系，合作机构覆盖国内主流券商、银行、信托、期货等各类金融机构①


User:  exit


Bye!


In [60]:
# ====================== Gradio UI ======================
import gradio as gr
import datetime as _dt
from typing import Dict, List  # ← 补上

DEFAULT_INDEX = "dd_report_data"

from typing import Dict, List

# 1) Chatbot 用 messages 模式
chatbot = gr.Chatbot(type="messages", height=420, show_copy_button=True, label="对话")

# 2) 历史转换：若已经是 messages，直接返回；否则从 (u, a) 转
def _history_to_dialogue(chat_history: list) -> List[Dict[str, str]]:
    if chat_history and isinstance(chat_history[0], dict):  # messages 模式
        return chat_history
    conv = []
    for u, a in chat_history:
        if u: conv.append({"role": "user", "content": u})
        if a: conv.append({"role": "assistant", "content": a})
    return conv

# 3) 追加对话：messages 需要两条 message
def _append(chat_history, user, assistant):
    if chat_history and isinstance(chat_history[0], dict):
        return chat_history + [
            {"role":"user","content":user},
            {"role":"assistant","content":assistant},
        ]
    return chat_history + [(user, assistant)]


def _format_hits_md(resp, topn: int = 8, *, show_full: bool = False, snippet_chars: int = 4096) -> str:
    """
    show_full=True  -> 显示全文
    show_full=False -> 只显示前 snippet_chars 个字符（默认 800）
    """
    if not resp or "hits" not in resp or not resp["hits"]["hits"]:
        return "（无命中）"
    lines = []
    for i, h in enumerate(resp["hits"]["hits"][:topn], 1):
        sc = h.get("_score", 0.0)
        src = h.get("_source", {})
        sec = src.get("section") or src.get("section_text") or "—"
        txt = (src.get("chunk") or "").strip().replace("\r\n", "\n")
        if not show_full and len(txt) > snippet_chars:
            body = txt[:snippet_chars] + "…"
        else:
            body = txt
        # 用 <details> 折叠（Gradio Markdown 支持基础 HTML，兼容长文）
        html = (
            f"<details open><summary><b>{i:02d}. [{sc:.3f}] {sec}</b></summary>"
            f"<div style='white-space:pre-wrap'>{body}</div></details>"
        )
        lines.append(html)
    return "\n\n".join(lines)


def respond(message, chat_history, temperature, n_ctx, index_name,
            show_full_hits, snippet_chars, answer_len):
    # 1) 历史转换 + 分类
    conversation = _history_to_dialogue(chat_history)
    history_text = "\n".join([f"{m['role']}: {m['content']}" for m in conversation[-6:]])
    label = classify_query(history_text, message) or "（未识别）"

    if "金融" not in label:
        assistant = "对不起，我只能处理金融相关问题。"
        chat_history = _append(chat_history, message, assistant)
        debug_md = f"**分类**：{label}  \n**时间**：{_dt.datetime.now():%Y-%m-%d %H:%M:%S}"
        return chat_history, "（无检索）", debug_md

    # 2) 改写
    rewrite, keywords, _raw = rewrite_query(conversation + [{"role": "user", "content": message}], message)

    # 3) 检索
    try:
        resp = es_hybrid_search(
            rewrite_text=rewrite,
            raw_text=message,
            index_name=index_name or DEFAULT_INDEX,
            size=int(n_ctx),
        )
        kb_ctx = [h["_source"].get("chunk", "") for h in resp["hits"]["hits"]]
        hits_md = _format_hits_md(resp, topn=min(8, int(n_ctx)),
                                  show_full=bool(show_full_hits), snippet_chars=int(snippet_chars))
    except Exception as e:
        kb_ctx, hits_md = [], f"检索失败：`{repr(e)}`"

    # 4) （可选）重排序
    merged = kb_ctx
    top_docs = rerank_documents(rewrite, merged, top_n=min(10, len(merged))) if merged else []
    passages_for_answer = [d["content"] for d in top_docs] if top_docs else merged

    # 5) 生成（把温度 + 最大长度传进去）
    temp = float(temperature) if temperature is not None else 0.2
    answer = generate_answer(passages_for_answer[:5], query=rewrite,
                             temperature=temp, max_new_tokens=int(answer_len))
    assistant = answer or "我不确定。"
    chat_history = _append(chat_history, message, assistant)

    # 6) 调试信息
    debug_md = (
        f"**分类**：{label}\n\n"
        f"**原始问题**：{message}\n\n"
        f"**改写后**：{rewrite}\n\n"
        f"**关键词**：{', '.join(keywords) if keywords else '—'}\n\n"
        f"**索引**：{index_name or DEFAULT_INDEX}\n\n"
        f"**温度**：{temp} | **答案上限**：{int(answer_len)} tokens\n\n"
        f"**时间**：{_dt.datetime.now():%Y-%m-%d %H:%M:%S}"
    )
    return chat_history, hits_md, debug_md


# ====== UI ======
with gr.Blocks(title="尽调问答助手", theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 尽调问答助手\n输入问题，我会基于本地知识库检索并回答。")

    with gr.Row():
        chatbot = gr.Chatbot(height=420, show_copy_button=True, label="对话")
        with gr.Column():
            hits = gr.Markdown(label="命中文档（Top）", value="（等待检索…）")
            dbg  = gr.Markdown(label="调试信息", value="—")

    with gr.Row():
        msg = gr.Textbox(placeholder="例如：公司简介 / 风控团队职责 / 交易系统架构…", scale=6)
        send = gr.Button("发送", variant="primary", scale=1)
        clear = gr.Button("清空", scale=1)

    with gr.Accordion("高级设置", open=False):
        temperature = gr.Slider(0.0, 1.5, value=0.2, step=0.05, label="生成温度")
        n_ctx = gr.Slider(3, 30, value=20, step=1, label="召回条数")
        index_name = gr.Textbox(value=DEFAULT_INDEX, label="ES 索引名")
        show_full_hits = gr.Checkbox(value=False, label="展开命中文档全文")
        snippet_chars = gr.Slider(200, 4000, value=800, step=50, label="命中文档截断长度（字符）")
        answer_len = gr.Slider(256, 2048, value=1024, step=64, label="答案最大长度（tokens）")


    def _submit(user_msg, chat_hist, temperature, n_ctx, index_name, show_full_hits, snippet_chars, answer_len):
        return respond(user_msg, chat_hist, temperature, int(n_ctx), index_name,
                   show_full_hits, snippet_chars, answer_len)

    send.click(_submit, [msg, chatbot, temperature, n_ctx, index_name, show_full_hits, snippet_chars, answer_len],
                    [chatbot, hits, dbg]).then(lambda: "", None, msg)

    msg.submit(_submit, [msg, chatbot, temperature, n_ctx, index_name, show_full_hits, snippet_chars, answer_len],
                    [chatbot, hits, dbg]).then(lambda: "", None, msg)
    clear.click(lambda: ([], "（已清空）", "—"), None, [chatbot, hits, dbg])

# 作为脚本启动
if __name__ == "__main__":
    # 如果需要公网访问可传 share=True；服务器环境可改 server_name="0.0.0.0"
    demo.queue().launch(server_name="127.0.0.1", server_port=7862, share=False)



C:\Users\sxmxs\AppData\Local\Temp\ipykernel_9012\1205298531.py:119: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=420, show_copy_button=True, label="对话")


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


D:\anaconda\envs\es\Lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
D:\anaconda\envs\es\Lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: The vector functions of the form function(query, doc['field']) are deprecated, and the form function(query, 'field') should be used instead. For example, cosineSimilarity(query, doc['field']) is replaced by cosineSimilarity(query, 'field').
  warnings.warn(message, category=ElasticsearchWarning)
D:\anaconda\envs\es\Lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible

In [61]:
import gradio as gr
try:
    demo.close()      # 若你有 demo 这个变量
except NameError:
    pass
# 关掉当前进程里所有还在监听的 gradio 服务
try:
    gr.close_all()
except AttributeError:
    pass

Closing server running on port: 7862
